In [ ]:
import numpy as np
import pandas as pd
from time import time
import pprint
import joblib
from functools import partial

import warnings
warnings.filterwarnings("ignore")

import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split

from skopt import BayesSearchCV
from skopt.callbacks import DeadlineStopper, DeltaYStopper
from skopt.space import Real, Integer
from scipy import stats

# Load data
train_data = pd.read_parquet('/kaggle/input/crunch/train_data.parquet')
test_data = pd.read_parquet('/kaggle/input/crunch/test_df.parquet')

**BayesSearchCV**

In [ ]:
# Define top features for each target
# top_feature_dict = {
#     'target_w': ['vratios_Feature_46', 'ekinetic_Feature_10', 'cmechanics_Feature_18', 'cmechanics_Feature_16', 'vratios_Feature_111', 'cmechanics_Feature_15', 'wrythm_Feature_146', 'cmechanics_Feature_17', 'wrythm_Feature_122', 'wrythm_Feature_98', 'wrythm_Feature_2', 'cmechanics_Feature_19', 'vratios_Feature_2', 'wrythm_Feature_50', 'vratios_Feature_47', 'wrythm_Feature_26', 'vratios_Feature_5', 'wrythm_Feature_170', 'cmechanics_Feature_14', 'vratios_Feature_3', 'ekinetic_Feature_23', 'wrythm_Feature_74', 'cmechanics_Feature_13', 'vratios_Feature_6', 'vratios_Feature_4', 'ekinetic_Feature_24', 'vratios_Feature_29', 'ekinetic_Feature_9', 'ekinetic_Feature_31', 'vratios_Feature_51'],
#     'target_r': ['vratios_Feature_46', 'vratios_Feature_2', 'vratios_Feature_29', 'vratios_Feature_47', 'vratios_Feature_3', 'vratios_Feature_5', 'wrythm_Feature_185', 'vratios_Feature_83', 'vratios_Feature_4', 'vratios_Feature_84', 'vratios_Feature_95', 'vratios_Feature_39', 'vratios_Feature_42', 'vratios_Feature_96', 'vratios_Feature_6', 'wrythm_Feature_162', 'vratios_Feature_51', 'wrythm_Feature_138', 'wrythm_Feature_42', 'vratios_Feature_63', 'wrythm_Feature_50', 'wrythm_Feature_186', 'vratios_Feature_94', 'wrythm_Feature_114', 'wrythm_Feature_90', 'vratios_Feature_45', 'wrythm_Feature_170', 'wrythm_Feature_146'],
#     'target_g': ['vratios_Feature_46', 'vratios_Feature_29', 'vratios_Feature_83', 'vratios_Feature_96', 'vratios_Feature_95', 'vratios_Feature_84', 'vratios_Feature_94', 'vratios_Feature_4', 'vratios_Feature_45', 'vratios_Feature_3', 'vratios_Feature_2', 'vratios_Feature_42', 'vratios_Feature_5', 'vratios_Feature_39', 'wrythm_Feature_185', 'vratios_Feature_48', 'vratios_Feature_47', 'vratios_Feature_63', 'vratios_Feature_32', 'wrythm_Feature_162', 'wrythm_Feature_66', 'wrythm_Feature_138', 'wrythm_Feature_10', 'vratios_Feature_51', 'wrythm_Feature_102', 'wrythm_Feature_126', 'wrythm_Feature_130', 'wrythm_Feature_54', 'wrythm_Feature_34'],
#     'target_b': ['vratios_Feature_46', 'vratios_Feature_29', 'vratios_Feature_96', 'vratios_Feature_45', 'vratios_Feature_83', 'vratios_Feature_4', 'vratios_Feature_95', 'vratios_Feature_84', 'vratios_Feature_3', 'vratios_Feature_94', 'vratios_Feature_2', 'vratios_Feature_5', 'vratios_Feature_42', 'wrythm_Feature_185', 'wrythm_Feature_66', 'wrythm_Feature_162', 'vratios_Feature_48', 'vratios_Feature_47', 'vratios_Feature_63', 'wrythm_Feature_138', 'vratios_Feature_39', 'wrythm_Feature_42', 'wrythm_Feature_106', 'wrythm_Feature_10', 'wrythm_Feature_130', 'wrythm_Feature_114', 'wrythm_Feature_90', 'wrythm_Feature_34', 'vratios_Feature_32', 'wrythm_Feature_186']
# }
top_feature_dict = {
    'target_w': ['3b1-signal_Feature_23', 'ekinetic_Feature_21', 'vratios_Feature_115', 'ekinetic_Feature_14', 'wrythm_Feature_86', 'dolly_Feature_27', 'dolly_Feature_29', 'cmechanics_Feature_7', 'wrythm_Feature_182', 'wrythm_Feature_6', 'wrythm_Feature_102', '3b1-signal_Feature_18', 'wrythm_Feature_126', 'cmechanics_Feature_86', 'wrythm_Feature_24', 'vratios_Feature_44', 'wrythm_Feature_30', 'wrythm_Feature_153', 'wrythm_Feature_33', 'wrythm_Feature_90'],
    'target_r': ['wrythm_Feature_186', 'dolly_Feature_12', 'wrythm_Feature_90', 'wrythm_Feature_162', 'dolly_Feature_13', 'wrythm_Feature_42', 'wrythm_Feature_66', 'wrythm_Feature_114', 'wrythm_Feature_138', 'dolly_Feature_27', 'wrythm_Feature_18', 'cmechanics_Feature_81', 'cmechanics_Feature_87', 'cmechanics_Feature_78', 'cmechanics_Feature_91', 'cmechanics_Feature_89', 'cmechanics_Feature_80', 'cmechanics_Feature_82', 'cmechanics_Feature_79', 'cmechanics_Feature_76'],
    'target_g': ['wrythm_Feature_186', 'wrythm_Feature_90', 'wrythm_Feature_162', 'wrythm_Feature_42', 'wrythm_Feature_114', 'wrythm_Feature_138', 'wrythm_Feature_66', 'wrythm_Feature_18', 'cmechanics_Feature_91', 'cmechanics_Feature_87', 'cmechanics_Feature_89', 'cmechanics_Feature_80', 'cmechanics_Feature_81', 'cmechanics_Feature_85', 'dolly_Feature_12', 'cmechanics_Feature_82', 'dolly_Feature_27', 'cmechanics_Feature_78', 'cmechanics_Feature_83', 'wrythm_Feature_178'],
    'target_b': ['wrythm_Feature_186', 'wrythm_Feature_114', 'wrythm_Feature_162', 'wrythm_Feature_138', 'wrythm_Feature_42', 'wrythm_Feature_90', 'wrythm_Feature_18', 'wrythm_Feature_66', 'cmechanics_Feature_91', 'cmechanics_Feature_89', 'cmechanics_Feature_87', 'cmechanics_Feature_80', 'cmechanics_Feature_81', 'cmechanics_Feature_85', 'dolly_Feature_12', 'cmechanics_Feature_82', 'cmechanics_Feature_83', 'dolly_Feature_27', 'wrythm_Feature_178', 'cmechanics_Feature_79']
}

def report_perf(optimizer, X, y, title="model", callbacks=None):
    start = time()
    
    if callbacks is not None:
        optimizer.fit(X, y, callback=callbacks)
    else:
        optimizer.fit(X, y)
        
    d=pd.DataFrame(optimizer.cv_results_)
    best_score = optimizer.best_score_
    best_score_std = d.iloc[optimizer.best_index_].std_test_score
    best_params = optimizer.best_params_
    
    print((title + " use %.2f seconds,  hyperparameters checked: %d, best CV score: %.3f "+ u"\u00B1"+" %.3f") % (time() - start, 
                                   len(optimizer.cv_results_['params']),
                                   best_score,
                                   best_score_std))    
    print('Best parameters:')
    pprint.pprint(best_params)
    print()
    return best_params

def optimize_target(train_data, test_data, target, top_feature_dict):
    features = [f for f in top_feature_dict[target] if f in train_data.columns]
    X_train = train_data[features]
    y_train = train_data[target]
    X_val = test_data[features]
    y_val = test_data[target]

    scoring = make_scorer(partial(mean_squared_error, squared=False), greater_is_better=False)
    
    reg = lgb.LGBMRegressor(boosting_type='gbdt',
                            metric='rmse',
                            objective='regression',
                            n_jobs=1, 
                            verbose=-1,
                            random_state=0)
    
    search_spaces = {
        'learning_rate': Real(0.01, 1.0, 'log-uniform'),     
        'n_estimators': Integer(30, 5000),                   
        'num_leaves': Integer(2, 512),                       
        'max_depth': Integer(1,16),                       
        'min_child_samples': Integer(1, 256),                
        'max_bin': Integer(100, 1000),                       
        'subsample': Real(0.01, 1.0, 'uniform'),             
        'subsample_freq': Integer(0, 10),                    
        'colsample_bytree': Real(0.01, 1.0, 'uniform'),      
        'min_child_weight': Real(0.01, 10.0, 'uniform'),     
        'reg_lambda': Real(1e-9, 100.0, 'log-uniform'),      
        'reg_alpha': Real(1e-9, 100.0, 'log-uniform'),       
    }
    
    opt = BayesSearchCV(estimator=reg,                                    
                        search_spaces=search_spaces,                      
                        scoring=scoring,                                  
                        cv=[(np.arange(len(y_train)), np.arange(len(y_train), len(y_train) + len(y_val)))], # Custom CV with train/val split
                        n_iter=60,                                       
                        n_points=3,                                       
                        n_jobs=-1,                                        
                        iid=False,                                        
                        return_train_score=False,                         
                        refit=False,                                      
                        optimizer_kwargs={'base_estimator': 'GP'},        
                        random_state=0)  
    
    overdone_control = DeltaYStopper(delta=0.0001)               
    time_limit_control = DeadlineStopper(total_time=60 * 60 * 1) 

    X_combined = np.vstack([X_train, X_val])
    y_combined = np.hstack([y_train, y_val])

    best_params = report_perf(opt, X_combined, y_combined, 'LGB', callbacks=[overdone_control, time_limit_control])

    return best_params


best_params_dict = {}
for target in ['target_w', 'target_r', 'target_g', 'target_b']:
    print(f'Optimizing {target}')
    best_params_dict[target] = optimize_target(train_data, test_data, target, top_feature_dict)

print("Best parameters for all targets:")
print(best_params_dict)


**GridSearchCV**

In [19]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error


train_df = pd.read_parquet('D:\\dataset\\dataset\\train_data.parquet')
test_df = pd.read_parquet('D:\\dataset\\dataset\\test_df.parquet')


X_train = train_df.drop(columns=['id', 'Moons', 'target_w', 'target_r', 'target_g', 'target_b'])
y_train_w = train_df['target_w']
y_train_r = train_df['target_r']
y_train_g = train_df['target_g']
y_train_b = train_df['target_b']

X_test = test_df.drop(columns=['id', 'Moons', 'target_w', 'target_r', 'target_g', 'target_b'])


gbm = lgb.LGBMRegressor()


param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'num_leaves': [31, 50, 100],
    'max_depth': [-1, 10, 16],
    'min_child_samples': [20, 30, 40],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'reg_alpha': [0, 0.1, 0.5],
    'reg_lambda': [0, 0.1, 0.5]
}


grid = GridSearchCV(estimator=gbm, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3, verbose=2, n_jobs=1)


grid.fit(X_train, y_train_w)
print('Best parameters for target_w:', grid.best_params_)


best_model = grid.best_estimator_
y_pred = best_model.predict(X_test)


Fitting 3 folds for each of 6561 candidates, totalling 19683 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.585335 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4305
[LightGBM] [Info] Number of data points in the train set: 160467, number of used features: 615
[LightGBM] [Info] Start training from score 0.499987
[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=-1, min_child_samples=20, num_leaves=31, reg_alpha=0, reg_lambda=0, subsample=0.7; total time=  23.5s
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.531988 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4305
[LightGBM] [Info] Number of data points in the train set: 160467, number of used features: 615
[LightGBM] [Info] Start training from score 0.499984
[CV] END colsample_bytree=0.7, learning_rate=0.01, max_depth=-1, min_child_samples=

**optuna**

In [ ]:
import optuna
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, GroupKFold
import numpy as np
from scipy import stats
import joblib
import pandas as pd

def xgb_optimize(trial, train_data, target, top_feature_dict):
    # Prepare the features and target
    features = [f for f in top_feature_dict[target] if f in train_data.columns]
    X = train_data[features]
    y = train_data[target]
    
    # Split the data
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Convert to numpy arrays
    X_train_np = X_train.values
    X_val_np = X_val.values
    y_train_np = y_train.values
    y_val_np = y_val.values
    
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 7e-4, 3e-1),
        'max_depth': trial.suggest_int('max_depth', 2, 13),
        'subsample': trial.suggest_float('subsample', 0.3, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.2, 1.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-4, 50, log=True),
        'grow_policy': trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),
        'max_cat_to_onehot': trial.suggest_int('max_cat_to_onehot', 4, 64),
        'n_estimators': 10000,
        'objective': 'reg:squarederror',
        'tree_method': 'hist',
        'random_state': 0,
        'enable_categorical': True,
    }

    gtss = GroupKFold(n_splits=5)
    scores = []

    model = XGBRegressor(**params)


    evals = [(X_val, y_val)]
    best_score = float('inf')
    best_iteration = 0

    for i in range(params['n_estimators']):
        model.set_params(n_estimators=i+1)
        model.fit(X_train, y_train, eval_set=evals, verbose=0)
        val_score = model.score(X_val, y_val)
        if val_score < best_score:
            best_score = val_score
            best_iteration = i + 1

    model.set_params(n_estimators=best_iteration)
    model.fit(X_train, y_train, eval_set=evals, verbose=0)

    yhat = model.predict(X_val)
    score = stats.spearmanr(y_val, yhat).correlation
    print(f'Fold score: {score}')
    scores.append(score)

    mean_scores = np.mean(scores)
    return mean_scores

def optimize_target(train_data, target, top_feature_dict, n_trials=50):
    features = [f for f in top_feature_dict[target] if f in train_data.columns]
    X = train_data[features]
    y = train_data[target]
    
 
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    
    X_train_np = X_train.values
    X_val_np = X_val.values
    y_train_np = y_train.values
    y_val_np = y_val.values
    
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: xgb_optimize(trial, train_data, target, top_feature_dict), n_trials=n_trials)
    
    best_params = study.best_params
    joblib.dump(best_params, f'best_params_{target}.pkl')
    
    print(f"Best parameters for {target}:")
    print(best_params)
    return best_params


train_data = pd.read_parquet('D:\\dataset\\dataset\\train_data.parquet')


top_feature_dict = {
    'target_w': [
        'vratios_Feature_46', 'ekinetic_Feature_10', 'cmechanics_Feature_18', 'cmechanics_Feature_16', 
        'vratios_Feature_111', 'cmechanics_Feature_15', 'wrythm_Feature_146', 'cmechanics_Feature_17', 
        'wrythm_Feature_122', 'wrythm_Feature_98', 'wrythm_Feature_2', 'cmechanics_Feature_19', 
        'vratios_Feature_2', 'wrythm_Feature_50', 'vratios_Feature_47', 'wrythm_Feature_26', 
        'vratios_Feature_5', 'wrythm_Feature_170', 'cmechanics_Feature_14', 'vratios_Feature_3', 
        'ekinetic_Feature_23', 'wrythm_Feature_74', 'cmechanics_Feature_13', 'vratios_Feature_6', 
        'vratios_Feature_4', 'ekinetic_Feature_24', 'vratios_Feature_29', 'ekinetic_Feature_9', 
        'ekinetic_Feature_31', 'vratios_Feature_51', 'vratios_Feature_95', 'vratios_Feature_83', 
        'ekinetic_Feature_17', '3b1-signal_Feature_23', 'cmechanics_Feature_12', 'wrythm_Feature_185', 
        'ekinetic_Feature_37', 'vratios_Feature_42', 'cmechanics_Feature_8', 'vratios_Feature_84', 
        'cmechanics_Feature_7', 'wrythm_Feature_151', 'wrythm_Feature_79', '3b1-signal_Feature_41', 
        'cmechanics_Feature_9', 'wrythm_Feature_7', 'wrythm_Feature_55', 'vratios_Feature_39', 
        'wrythm_Feature_127', 'vratios_Feature_96', 'ekinetic_Feature_32', 'ekinetic_Feature_16', 
        'wrythm_Feature_103', 'wrythm_Feature_31', 'cmechanics_Feature_11', 'wrythm_Feature_175', 
        'cmechanics_Feature_36', 'wrythm_Feature_42', 'vratios_Feature_32', 'wrythm_Feature_66', 
        'wrythm_Feature_162', 'vratios_Feature_63', 'wrythm_Feature_138', 'vratios_Feature_64', 
        'ekinetic_Feature_36', 'cmechanics_Feature_10', 'ekinetic_Feature_38', 'wrythm_Feature_115', 
        'ekinetic_Feature_22', 'vratios_Feature_45', 'ekinetic_Feature_15', 'vratios_Feature_31', 
        'vratios_Feature_94', 'cmechanics_Feature_84', 'vratios_Feature_1', '3b1-signal_Feature_16', 
        'wrythm_Feature_18', 'wrythm_Feature_90', 'wrythm_Feature_186', 'wrythm_Feature_171', 
        'vratios_Feature_55', 'vratios_Feature_33', 'wrythm_Feature_114', 'ekinetic_Feature_25', 
        'wrythm_Feature_65', 'vratios_Feature_38', 'wrythm_Feature_147', '3b1-signal_Feature_48', 
        'vratios_Feature_35', 'wrythm_Feature_51', 'vratios_Feature_7', 'wrythm_Feature_163', 
        'wrythm_Feature_139', 'ekinetic_Feature_30', 'vratios_Feature_36', 'wrythm_Feature_54', 
        'dolly_Feature_17', 'wrythm_Feature_30', 'wrythm_Feature_123', 'wrythm_Feature_12', 
        'wrythm_Feature_27', 'vratios_Feature_11', 'wrythm_Feature_150', 'ekinetic_Feature_33', 
        'cmechanics_Feature_73', 'vratios_Feature_79', 'vratios_Feature_93', 'cmechanics_Feature_20', 
        'vratios_Feature_37', 'wrythm_Feature_75', 'wrythm_Feature_84', 'wrythm_Feature_156', 
        'wrythm_Feature_99', 'vratios_Feature_14', 'wrythm_Feature_192', 'wrythm_Feature_126', 
        'vratios_Feature_112', 'vratios_Feature_48', 'wrythm_Feature_108', 'vratios_Feature_116', 
        'wrythm_Feature_3', 'wrythm_Feature_161', 'vratios_Feature_9', 'wrythm_Feature_106', 
        'cmechanics_Feature_6', 'wrythm_Feature_132', '3b1-signal_Feature_28', '3b1-signal_Feature_50', 
        'wrythm_Feature_102', 'wrythm_Feature_36'
    ],
    'target_r': [
        'vratios_Feature_46', 'vratios_Feature_2', 'vratios_Feature_29', 'vratios_Feature_47', 
        'vratios_Feature_3', 'vratios_Feature_5', 'wrythm_Feature_185', 'vratios_Feature_83', 
        'vratios_Feature_4', 'vratios_Feature_84', 'vratios_Feature_95', 'vratios_Feature_39', 
        'vratios_Feature_42', 'vratios_Feature_96', 'vratios_Feature_6', 'wrythm_Feature_162', 
        'vratios_Feature_32', 'wrythm_Feature_66', 'vratios_Feature_51', 'wrythm_Feature_138', 
        'wrythm_Feature_42', 'vratios_Feature_63', 'wrythm_Feature_50', 'wrythm_Feature_186', 
        'vratios_Feature_94', 'wrythm_Feature_114', 'wrythm_Feature_90', 'vratios_Feature_45', 
        'wrythm_Feature_170', 'wrythm_Feature_146', 'wrythm_Feature_79', 'ekinetic_Feature_9', 
        'wrythm_Feature_122', 'cmechanics_Feature_13', 'cmechanics_Feature_14', 'wrythm_Feature_18', 
        'wrythm_Feature_74', 'vratios_Feature_93', 'wrythm_Feature_151', 'wrythm_Feature_55', 
        'wrythm_Feature_26', 'vratios_Feature_33', 'cmechanics_Feature_15', 'vratios_Feature_64', 
        'vratios_Feature_36', 'vratios_Feature_31', 'vratios_Feature_35', 'vratios_Feature_1', 
        'vratios_Feature_48', 'wrythm_Feature_31', 'wrythm_Feature_127', 'wrythm_Feature_98', 
        'vratios_Feature_37', 'wrythm_Feature_7', 'cmechanics_Feature_9', 'wrythm_Feature_175', 
        'cmechanics_Feature_16', 'wrythm_Feature_2', 'wrythm_Feature_106', 'wrythm_Feature_103', 
        'wrythm_Feature_10', 'wrythm_Feature_161', 'wrythm_Feature_19', 'vratios_Feature_38', 
        'wrythm_Feature_130', 'cmechanics_Feature_11', 'cmechanics_Feature_18', 'wrythm_Feature_34', 
        'wrythm_Feature_58', 'cmechanics_Feature_8', 'cmechanics_Feature_12', 'vratios_Feature_11', 
        'vratios_Feature_14', 'cmechanics_Feature_17', 'vratios_Feature_7', 'wrythm_Feature_64', 
        'wrythm_Feature_82', 'vratios_Feature_76', 'wrythm_Feature_24', 'wrythm_Feature_57', 
        'vratios_Feature_51', 'cmechanics_Feature_10', 'wrythm_Feature_78', 'wrythm_Feature_8', 
        'vratios_Feature_4', 'vratios_Feature_77', 'wrythm_Feature_107', 'ekinetic_Feature_33', 
        'ekinetic_Feature_34', 'wrythm_Feature_52', 'wrythm_Feature_128', 'vratios_Feature_49', 
        'wrythm_Feature_146', 'wrythm_Feature_60', 'cmechanics_Feature_20', 'wrythm_Feature_5', 
        'wrythm_Feature_124', 'wrythm_Feature_184', 'wrythm_Feature_11', 'wrythm_Feature_33', 
        'wrythm_Feature_72'
    ],
    'target_g': [
        'cmechanics_Feature_18', 'vratios_Feature_46', 'wrythm_Feature_131', 'cmechanics_Feature_17', 
        'wrythm_Feature_102', 'wrythm_Feature_36', 'wrythm_Feature_37', 'wrythm_Feature_170', 
        'wrythm_Feature_66', 'wrythm_Feature_126', 'wrythm_Feature_63', 'wrythm_Feature_16', 
        'wrythm_Feature_149', 'wrythm_Feature_24', 'wrythm_Feature_44', 'wrythm_Feature_23', 
        'wrythm_Feature_20', 'wrythm_Feature_1', 'wrythm_Feature_61', 'wrythm_Feature_79', 
        'wrythm_Feature_54', 'wrythm_Feature_170', 'wrythm_Feature_121', 'wrythm_Feature_70', 
        'wrythm_Feature_80', 'wrythm_Feature_132', 'wrythm_Feature_96', 'wrythm_Feature_125', 
        'wrythm_Feature_134', 'wrythm_Feature_155', 'wrythm_Feature_21', 'wrythm_Feature_164', 
        'wrythm_Feature_75', 'wrythm_Feature_51', 'wrythm_Feature_115', 'wrythm_Feature_65', 
        'wrythm_Feature_63', 'wrythm_Feature_117', 'wrythm_Feature_138', 'wrythm_Feature_105', 
        'wrythm_Feature_118', 'wrythm_Feature_123', 'wrythm_Feature_138', 'wrythm_Feature_57', 
        'wrythm_Feature_148', 'wrythm_Feature_73', 'wrythm_Feature_94', 'wrythm_Feature_52', 
        'wrythm_Feature_26', 'wrythm_Feature_85', 'wrythm_Feature_143', 'wrythm_Feature_167', 
        'wrythm_Feature_140', 'wrythm_Feature_90', 'wrythm_Feature_109', 'wrythm_Feature_43', 
        'wrythm_Feature_19', 'wrythm_Feature_74', 'wrythm_Feature_55', 'wrythm_Feature_6', 
        'wrythm_Feature_57', 'wrythm_Feature_159', 'wrythm_Feature_16', 'wrythm_Feature_56', 
        'wrythm_Feature_148', 'wrythm_Feature_167', 'wrythm_Feature_167', 'wrythm_Feature_112', 
        'wrythm_Feature_146', 'wrythm_Feature_30', 'wrythm_Feature_99', 'wrythm_Feature_84', 
        'wrythm_Feature_46', 'wrythm_Feature_44', 'wrythm_Feature_109', 'wrythm_Feature_118', 
        'wrythm_Feature_118', 'wrythm_Feature_153', 'wrythm_Feature_149', 'wrythm_Feature_124', 
        'wrythm_Feature_31', 'wrythm_Feature_116', 'wrythm_Feature_98', 'wrythm_Feature_146', 
        'wrythm_Feature_99', 'wrythm_Feature_46', 'wrythm_Feature_26', 'wrythm_Feature_164', 
        'wrythm_Feature_41', 'wrythm_Feature_52', 'wrythm_Feature_27'
    ],
    'target_b': [
        'vratios_Feature_29', 'wrythm_Feature_98', 'vratios_Feature_47', 'vratios_Feature_37', 
        'vratios_Feature_95', 'vratios_Feature_5', 'vratios_Feature_84', 'vratios_Feature_6', 
        'vratios_Feature_35', 'vratios_Feature_39', 'wrythm_Feature_161', 'wrythm_Feature_43', 
        'wrythm_Feature_145', 'wrythm_Feature_124', 'wrythm_Feature_160', 'vratios_Feature_1', 
        'wrythm_Feature_147', 'wrythm_Feature_137', 'wrythm_Feature_70', 'wrythm_Feature_43', 
        'wrythm_Feature_144', 'wrythm_Feature_59', 'wrythm_Feature_141', 'wrythm_Feature_63', 
        'vratios_Feature_48', 'wrythm_Feature_153', 'vratios_Feature_46', 'wrythm_Feature_148', 
        'wrythm_Feature_26', 'wrythm_Feature_142', 'wrythm_Feature_110', 'wrythm_Feature_83', 
        'wrythm_Feature_116', 'wrythm_Feature_56', 'wrythm_Feature_104', 'vratios_Feature_4', 
        'wrythm_Feature_135', 'vratios_Feature_51', 'wrythm_Feature_26', 'wrythm_Feature_74', 
        'wrythm_Feature_108', 'wrythm_Feature_30', 'vratios_Feature_85', 'vratios_Feature_43', 
        'wrythm_Feature_139', 'vratios_Feature_94', 'vratios_Feature_41', 'wrythm_Feature_109', 
        'wrythm_Feature_125', 'wrythm_Feature_99', 'vratios_Feature_31', 'wrythm_Feature_100', 
        'vratios_Feature_33', 'wrythm_Feature_142', 'wrythm_Feature_76', 'vratios_Feature_24', 
        'wrythm_Feature_145', 'wrythm_Feature_90', 'vratios_Feature_35', 'wrythm_Feature_94', 
        'wrythm_Feature_110', 'vratios_Feature_36', 'wrythm_Feature_85', 'vratios_Feature_46', 
        'wrythm_Feature_161', 'vratios_Feature_53', 'wrythm_Feature_41', 'wrythm_Feature_25', 
        'wrythm_Feature_28', 'vratios_Feature_12', 'wrythm_Feature_81', 'wrythm_Feature_58', 
        'wrythm_Feature_145', 'vratios_Feature_10', 'wrythm_Feature_63', 'wrythm_Feature_116', 
        'wrythm_Feature_87', 'wrythm_Feature_46', 'wrythm_Feature_76', 'vratios_Feature_85'
    ]
}
for target in ['target_w', 'target_r', 'target_g', 'target_b']:
    best_params = optimize_target(train_data, target, top_feature_dict)
    joblib.dump(best_params, f'best_params_{target}.pkl')